In [1]:
import os
import sys
import numpy as np
import pandas as pd
import json
import random

In [2]:
train_file = '/data/sulixin/research/contest/quac/train_v0.2.json'
dev_file = '/data/sulixin/research/contest/quac/val_v0.2.json'

train = json.load(open(train_file))
dev = json.load(open(dev_file))

def basic_stat(data):
    print('paragraph {}'.format(len(data)))
    
basic_stat(train['data'])
basic_stat(dev['data'])

paragraph 11567
paragraph 1000


In [3]:
train['data'][10]

{'paragraphs': [{'context': 'Following the success of Hell\'s Angels, Thompson was able to publish articles in a number of well-known magazines during the late 1960s, including The New York Times Magazine, Esquire, Pageant, and Harper\'s. In the Times Magazine article, published in 1967, shortly before the "Summer of Love", and titled "The Hashbury is the Capital of the Hippies", Thompson wrote in-depth about the Hippies of San Francisco, deriding a culture that began to lack the political convictions of the New Left and the artistic core of the Beats, instead becoming overrun with newcomers lacking any purpose other than obtaining drugs. It was an observation on the 1960s\' counterculture that Thompson would further examine in Fear and Loathing in Las Vegas and other articles.  By late 1967, Thompson and his family moved back to Colorado and rented a house in Woody Creek, a small mountain hamlet outside Aspen. In early 1969, Thompson finally received a $15,000 royalty check for the pa

In [4]:
def avg(li):
    return sum(li) * 1.0 / len(li)

def avg_c_len(li):
    return avg( [len(d['context'].split()) for d in li] )

def avg_q_len(paras):
    return avg ( [len(qa['question'].split()) for para in paras for qa in para['qas']] )

def avg_a_len(paras):
    return avg ( [len(qa['answers'][0]['text'].split()) for para in paras for qa in para['qas']] )

def qa_cnt(paras):
    return avg ( [len(para['qas']) for para in paras] )

def a_diff(paras):
    return avg( [ int(qa['answers'][0]['text'] == qa['orig_answer']['text']) for para in paras for qa in para['qas']] )

def a_in_c(paras):
    return avg([qa['answers'][0]['text'] in p['context'] for p in paras for qa in p['qas']])

def yesno_rate(paras):
    return avg([qa['yesno']=='y' for p in paras for qa in p['qas']])

def my_stat(data):
    print()
    rv = {'context_len': [avg_c_len(x['paragraphs']) for x in data],\
          'para_cnt': [len(x['paragraphs']) for x in data],\
          'query_len': [avg_q_len(x['paragraphs']) for x in data],\
          'answer_len': [avg_a_len(x['paragraphs']) for x in data],\
          'qa_cnt': [qa_cnt(x['paragraphs']) for x in data],\
          'answer_diff': [a_diff(x['paragraphs']) for x in data],\
          'avg_bg_len': [len( x['background'].split()) for x in data],\
          'a_in_c': [a_in_c(x['paragraphs']) for x in data],
          'yesno_rate': [yesno_rate(x['paragraphs']) for x in data]
         }
    df = pd.DataFrame(rv)
    return df.describe()

In [5]:
my_stat(train['data'])

,context_len,para_cnt,query_len,answer_len,qa_cnt,answer_diff,avg_bg_len,a_in_c,yesno_rate
count,11567.000000,11567.0,11567.000000,11567.000000,11567.000000,11567.0,11567.000000,11567.0,11567.000000
mean,394.796490,1.0,6.513830,12.458906,7.224691,1.0,86.114982,1.0,0.199479
std,150.331075,0.0,1.222745,4.603039,2.109509,0.0,23.572367,0.0,0.206890
min,251.000000,1.0,2.888889,1.000000,4.000000,1.0,3.000000,1.0,0.000000
25%,307.000000,1.0,5.666667,9.250000,6.000000,1.0,70.000000,1.0,0.000000
50%,365.000000,1.0,6.375000,12.333333,7.000000,1.0,85.000000,1.0,0.166667
75%,443.000000,1.0,7.250000,15.500000,9.000000,1.0,100.000000,1.0,0.300000
max,1615.000000,1.0,15.000000,27.250000,12.000000,1.0,219.000000,1.0,1.000000


In [6]:
my_stat(dev['data'])

,context_len,para_cnt,query_len,answer_len,qa_cnt,answer_diff,avg_bg_len,a_in_c,yesno_rate
count,1000.000000,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.0,1000.000000
mean,440.971000,1.0,6.541135,11.826507,7.354000,0.342009,85.915000,1.0,0.199992
std,232.837556,0.0,1.197591,4.076733,2.119239,0.205411,22.938045,0.0,0.166908
min,251.000000,1.0,3.750000,1.000000,4.000000,0.000000,6.000000,1.0,0.000000
25%,312.750000,1.0,5.666667,9.106061,6.000000,0.200000,71.000000,1.0,0.083333
50%,379.000000,1.0,6.375000,11.707143,7.000000,0.333333,86.000000,1.0,0.181818
75%,465.250000,1.0,7.250000,14.555556,9.000000,0.500000,102.000000,1.0,0.285714
max,1584.000000,1.0,11.800000,28.000000,12.000000,1.000000,174.000000,1.0,0.800000
